# Introduction to AM-QADF Framework

## Purpose

This notebook provides an interactive introduction to the **AM-QADF** (Additive Manufacturing Quality Assessment and Data Fusion) framework. You'll explore the framework architecture, modules, capabilities, and key concepts through interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Understand the framework purpose and scope
- ✅ Know how to install and configure AM-QADF
- ✅ Understand core data structures (VoxelGrid, QueryResult, etc.)
- ✅ See the complete workflow at a high level
- ✅ Explore framework modules interactively

## Estimated Duration

30-45 minutes

---

## What is AM-QADF?

**AM-QADF** is a comprehensive Python framework for processing, analyzing, and visualizing multi-source additive manufacturing data. It provides a unified interface for:

- 🔍 **Querying** diverse data sources (hatching, laser, CT, ISPM)
- 🧊 **Voxelizing** point cloud data into structured 3D grids
- 🎯 **Mapping signals** to voxel domains using multiple interpolation methods
- ⏰ **Synchronizing** temporal and spatial data from multiple sources
- 🔀 **Fusing** multi-source data into unified representations
- ✅ **Assessing quality** of data and signals
- 📊 **Analyzing** data statistically and through sensitivity analysis
- 🚨 **Detecting anomalies** using multiple algorithms
- 🎨 **Visualizing** 3D voxel data interactively

---

## Interactive Framework Explorer

Use the widgets below to explore the framework interactively. No coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label
)
from IPython.display import display, Markdown, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Try to import AM-QADF
try:
    import am_qadf
    AM_QADF_AVAILABLE = True
    try:
        AM_QADF_VERSION = am_qadf.__version__
    except AttributeError:
        AM_QADF_VERSION = "1.0.0"
except ImportError:
    AM_QADF_AVAILABLE = False
    AM_QADF_VERSION = "Not Installed"

# Check dependencies
def check_dependency(module_name):
    """Check if a dependency is installed."""
    try:
        __import__(module_name)
        return True
    except ImportError:
        return False

DEPENDENCIES = {
    'numpy': check_dependency('numpy'),
    'pandas': check_dependency('pandas'),
    'scipy': check_dependency('scipy'),
    'pymongo': check_dependency('pymongo'),
    'matplotlib': check_dependency('matplotlib'),
    'pyvista': check_dependency('pyvista'),
    'ipywidgets': check_dependency('ipywidgets'),
}

# Check MongoDB connection (optional)
MONGODB_AVAILABLE = False
try:
    from src.infrastructure.database import get_connection_manager
    manager = get_connection_manager(env_name="development")
    mongodb_client = manager.get_mongodb_client()
    # Try to ping
    mongodb_client.admin.command('ping')
    MONGODB_AVAILABLE = True
except Exception:
    MONGODB_AVAILABLE = False

print("✅ Setup complete!")
print(f"AM-QADF Available: {AM_QADF_AVAILABLE}")
print(f"AM-QADF Version: {AM_QADF_VERSION}")
print(f"MongoDB Available: {MONGODB_AVAILABLE}")


Failed to connect to MongoDB: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 69600f1fdd146cd040d38c9e, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


Failed to initialize MongoDB connection: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 69600f1fdd146cd040d38c9e, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


✅ Setup complete!
AM-QADF Available: True
AM-QADF Version: 1.0.0
MongoDB Available: False


In [2]:
# Framework Module Information
MODULE_INFO = {
    'Core': {
        'description': 'Foundation layer with domain entities, value objects, and exceptions',
        'key_classes': ['VoxelData', 'VoxelCoordinates', 'QualityMetric', 'QueryResult'],
        'capabilities': [
            'Domain entities and value objects',
            'Exception handling',
            'Core data structures'
        ],
        'dependencies': []
    },
    'Query': {
        'description': 'Unified data warehouse query interface for multiple data sources',
        'key_classes': ['UnifiedQueryClient', 'HatchingClient', 'LaserClient', 'CTClient', 'ISPMClient'],
        'capabilities': [
            'Query hatching paths',
            'Query laser parameters',
            'Query CT scan data',
            'Query ISPM sensor data',
            'Spatial and temporal filtering',
            'Multi-source queries'
        ],
        'dependencies': ['Core']
    },
    'Voxelization': {
        'description': 'Voxel grid creation and management for 3D spatial data',
        'key_classes': ['VoxelGrid', 'CoordinateSystem', 'MultiResolutionGrid', 'AdaptiveResolutionGrid'],
        'capabilities': [
            'Create uniform voxel grids',
            'Create adaptive resolution grids',
            'Create multi-resolution grids',
            'Coordinate system transformations',
            'Grid properties and metadata'
        ],
        'dependencies': ['Core']
    },
    'Signal Mapping': {
        'description': '⭐ CRITICAL - Interpolate signals onto voxel grids using multiple methods',
        'key_classes': ['NearestNeighbor', 'LinearInterpolation', 'IDWInterpolation', 'KDEInterpolation'],
        'capabilities': [
            'Nearest neighbor interpolation',
            'Linear interpolation',
            'Inverse Distance Weighting (IDW)',
            'Kernel Density Estimation (KDE)',
            'Parallel execution',
            'Spark-based distributed processing'
        ],
        'dependencies': ['Voxelization', 'Core']
    },
    'Synchronization': {
        'description': 'Temporal and spatial alignment of multi-source data',
        'key_classes': ['TemporalAligner', 'SpatialTransformer', 'LayerTimeMapper', 'DataFusion'],
        'capabilities': [
            'Temporal alignment (time-to-layer mapping)',
            'Spatial coordinate transformations',
            'Multi-source data synchronization',
            'Alignment accuracy assessment'
        ],
        'dependencies': ['Signal Mapping', 'Voxelization', 'Core']
    },
    'Correction': {
        'description': 'Geometric distortion correction and calibration',
        'key_classes': ['ScalingModel', 'RotationModel', 'WarpingModel', 'CalibrationManager'],
        'capabilities': [
            'Scaling correction',
            'Rotation correction',
            'Warping correction',
            'Calibration data usage',
            'Correction validation'
        ],
        'dependencies': ['Synchronization', 'Core']
    },
    'Processing': {
        'description': 'Signal processing and noise reduction',
        'key_classes': ['OutlierDetector', 'SignalSmoother', 'NoiseReductionPipeline', 'ThermalFieldGenerator'],
        'capabilities': [
            'Outlier detection',
            'Signal smoothing',
            'Noise reduction',
            'Derived signal generation (thermal, density, stress)'
        ],
        'dependencies': ['Synchronization', 'Core']
    },
    'Fusion': {
        'description': 'Multi-source data fusion strategies',
        'key_classes': ['MultiVoxelGridFusion', 'WeightedAverageFusion', 'QualityBasedFusion', 'MedianFusion'],
        'capabilities': [
            'Weighted average fusion',
            'Median fusion',
            'Quality-based fusion',
            'Max/Min fusion',
            'Fusion quality assessment'
        ],
        'dependencies': ['Synchronization', 'Core']
    },
    'Quality': {
        'description': 'Quality assessment and metrics',
        'key_classes': ['QualityAssessmentClient', 'CompletenessAnalyzer', 'SignalQualityAnalyzer', 'AlignmentAccuracyAnalyzer'],
        'capabilities': [
            'Data completeness analysis',
            'Signal quality metrics (SNR, uncertainty)',
            'Alignment accuracy assessment',
            'Overall data quality evaluation',
            'Gap detection and filling'
        ],
        'dependencies': ['Fusion', 'Core']
    },
    'Analytics': {
        'description': 'Statistical and sensitivity analysis',
        'key_classes': ['StatisticalAnalysisClient', 'SensitivityAnalysisClient', 'ProcessAnalyzer', 'VirtualExperimentClient'],
        'capabilities': [
            'Descriptive statistics',
            'Correlation analysis',
            'Trend analysis',
            'Sensitivity analysis (Sobol, Morris)',
            'Process analysis and optimization',
            'Virtual experiments'
        ],
        'dependencies': ['Fusion', 'Core']
    },
    'Anomaly Detection': {
        'description': 'Anomaly detection using multiple algorithms',
        'key_classes': ['AnomalyDetectionClient', 'StatisticalDetector', 'ClusteringDetector', 'MLDetector', 'EnsembleDetector'],
        'capabilities': [
            'Statistical detectors (Z-score, IQR, Mahalanobis)',
            'Clustering detectors (DBSCAN, Isolation Forest, LOF)',
            'ML-based detectors (Autoencoder, LSTM, VAE)',
            'Rule-based detectors',
            'Ensemble methods',
            'Voxel-level anomaly detection'
        ],
        'dependencies': ['Fusion', 'Core']
    },
    'Visualization': {
        'description': '3D visualization of voxel data',
        'key_classes': ['VoxelRenderer', 'MultiResolutionViewer', 'SliceViewer'],
        'capabilities': [
            '3D voxel grid rendering (PyVista)',
            'Multi-resolution visualization',
            'Slice visualization',
            'Volume rendering',
            'Animation and time-series visualization'
        ],
        'dependencies': ['Fusion', 'Core']
    },
    'Voxel Domain': {
        'description': 'Main orchestrator for complete workflows',
        'key_classes': ['VoxelDomainClient', 'VoxelStorage'],
        'capabilities': [
            'High-level workflow orchestration',
            'Automated processing pipelines',
            'Voxel grid storage and retrieval',
            'Batch processing',
            'Workflow management'
        ],
        'dependencies': ['All modules']
    }
}

# Key Concepts
KEY_CONCEPTS = {
    'VoxelGrid': {
        'definition': 'A 3D grid structure that divides space into regular cubic units (voxels) for storing spatial data',
        'usage': 'Used to represent manufacturing data in a unified 3D space'
    },
    'Signal Mapping': {
        'definition': 'The process of interpolating point-based data onto voxel grids using interpolation methods',
        'usage': 'Critical step that enables multi-source data fusion in voxel domain'
    },
    'Data Fusion': {
        'definition': 'Combining signals from multiple sources into a unified voxel representation',
        'usage': 'Enables comprehensive analysis by combining all available data sources'
    },
    'Quality Assessment': {
        'definition': 'Evaluating data quality, completeness, and alignment accuracy',
        'usage': 'Ensures reliable analysis results by validating data quality'
    },
    'Sensitivity Analysis': {
        'definition': 'Analyzing how process parameters affect output quality',
        'usage': 'Identifies key parameters for process optimization'
    }
}

# Data Structures
DATA_STRUCTURES = {
    'VoxelGrid': {
        'attributes': ['bbox_min', 'bbox_max', 'resolution', 'dims', 'voxels', 'available_signals'],
        'description': 'Core data structure for 3D spatial data representation',
        'example': 'grid = VoxelGrid(bbox_min=(0,0,0), bbox_max=(100,100,100), resolution=1.0)'
    },
    'QueryResult': {
        'attributes': ['data', 'metadata', 'source', 'spatial_bounds', 'temporal_range'],
        'description': 'Result container for data warehouse queries',
        'example': 'result = query_client.query_hatching(model_id="model1")'
    },
    'VoxelData': {
        'attributes': ['signals', 'coordinates', 'quality', 'metadata'],
        'description': 'Data container for individual voxel',
        'example': 'voxel_data = VoxelData(signals={"power": 200.0}, coordinates=(10, 20, 30))'
    }
}

# Common Workflows
WORKFLOWS = {
    'Complete Workflow': {
        'steps': [
            '1. Query data from warehouse',
            '2. Create voxel grid',
            '3. Map signals to voxels',
            '4. Align and synchronize',
            '5. Correct and process',
            '6. Fuse multi-source data',
            '7. Assess quality',
            '8. Perform analytics',
            '9. Detect anomalies',
            '10. Visualize results'
        ],
        'description': 'End-to-end workflow from data query to visualization'
    },
    'Quick Analysis': {
        'steps': [
            '1. Query data',
            '2. Create voxel grid',
            '3. Map signals',
            '4. Perform statistical analysis',
            '5. Visualize results'
        ],
        'description': 'Quick workflow for basic analysis'
    },
    'Quality Assessment': {
        'steps': [
            '1. Query data',
            '2. Create voxel grid',
            '3. Map signals',
            '4. Assess data quality',
            '5. Assess signal quality',
            '6. Generate quality report'
        ],
        'description': 'Workflow focused on quality assessment'
    }
}

print("✅ Framework information loaded!")


✅ Framework information loaded!


## Interactive Framework Explorer

Use the widgets below to explore the framework. Click on modules, select data structures, and explore workflows - all interactively!


In [3]:
# Create Interactive Widget Interface

# Top Panel: Framework Version, Quick Navigation, Theme
version_text = widgets.HTML(f"<b>AM-QADF Version:</b> {AM_QADF_VERSION}")
quick_nav = Dropdown(
    options=['Overview', 'Modules', 'Data Structures', 'Workflows', 'Installation'],
    value='Overview',
    description='Navigate:',
    style={'description_width': 'initial'},
    layout=Layout(width='200px')
)

theme_selector = RadioButtons(
    options=[('Light', 'light'), ('Dark', 'dark')],
    value='light',
    description='Theme:',
    style={'description_width': 'initial'}
)

top_panel = HBox([
    version_text,
    quick_nav,
    theme_selector
], layout=Layout(justify_content='space-between', padding='10px', border='1px solid #ccc'))

# Left Panel: Module Explorer
# Create module accordion items
module_widgets = []
for module_name in MODULE_INFO.keys():
    info = MODULE_INFO[module_name]
    
    # Description
    desc = widgets.HTML(f"<p><b>Description:</b> {info['description']}</p>")
    
    # Key Classes
    classes_dropdown = Dropdown(
        options=info['key_classes'],
        description='Key Classes:',
        style={'description_width': 'initial'}
    )
    
    # Capabilities
    capabilities_html = "<p><b>Capabilities:</b></p><ul>"
    for cap in info['capabilities']:
        capabilities_html += f"<li>{cap}</li>"
    capabilities_html += "</ul>"
    capabilities = widgets.HTML(capabilities_html)
    
    # Dependencies
    deps_text = ", ".join(info['dependencies']) if info['dependencies'] else "None (base module)"
    dependencies = widgets.HTML(f"<p><b>Dependencies:</b> {deps_text}</p>")
    
    module_widget = VBox([desc, classes_dropdown, capabilities, dependencies])
    module_widgets.append(module_widget)

# Create accordion with children and set titles
module_accordion = Accordion(children=tuple(module_widgets))
for i, module_name in enumerate(MODULE_INFO.keys()):
    module_accordion.set_title(i, module_name)

# Center Panel: Interactive Visualizations
center_output = Output(layout=Layout(height='600px', overflow='auto'))

def update_center_panel(nav_value):
    """Update center panel based on navigation selection."""
    with center_output:
        clear_output(wait=True)
        if nav_value == 'Overview':
            display(HTML("""
            <h3>Framework Architecture</h3>
            <p>The AM-QADF framework follows a modular, layered architecture:</p>
            <ul>
                <li><b>Core Layer:</b> Foundation with entities and value objects</li>
                <li><b>Data Layer:</b> Query and voxelization modules</li>
                <li><b>Processing Layer:</b> Signal mapping, synchronization, correction, processing</li>
                <li><b>Fusion Layer:</b> Multi-source data fusion</li>
                <li><b>Analysis Layer:</b> Quality, analytics, anomaly detection</li>
                <li><b>Output Layer:</b> Visualization and storage</li>
            </ul>
            <h3>Data Flow</h3>
            <p>The framework processes data through these stages:</p>
            <ol>
                <li>Query data from warehouse (multiple sources)</li>
                <li>Create voxel grid structure</li>
                <li>Map signals to voxels (interpolation)</li>
                <li>Align and synchronize multi-source data</li>
                <li>Correct geometric distortions</li>
                <li>Process and clean signals</li>
                <li>Fuse multi-source data</li>
                <li>Assess quality</li>
                <li>Perform analytics</li>
                <li>Detect anomalies</li>
                <li>Visualize results</li>
            </ol>
            """))
        elif nav_value == 'Modules':
            display(HTML("""
            <h3>Framework Modules</h3>
            <p>Select a module from the left panel to see details.</p>
            <p>The framework consists of 13 modules organized in a dependency hierarchy:</p>
            <ul>
                <li><b>Core:</b> Foundation (no dependencies)</li>
                <li><b>Query, Voxelization:</b> Data access and grid creation</li>
                <li><b>Signal Mapping:</b> Critical interpolation step</li>
                <li><b>Synchronization, Correction, Processing:</b> Data alignment and processing</li>
                <li><b>Fusion:</b> Multi-source data combination</li>
                <li><b>Quality, Analytics, Anomaly Detection, Visualization:</b> Analysis and output</li>
                <li><b>Voxel Domain:</b> Main orchestrator</li>
            </ul>
            """))
        elif nav_value == 'Data Structures':
            display(HTML("""
            <h3>Core Data Structures</h3>
            <p>Select a data structure from the right panel to see details.</p>
            <p>Key data structures in AM-QADF:</p>
            <ul>
                <li><b>VoxelGrid:</b> 3D grid structure for spatial data</li>
                <li><b>QueryResult:</b> Container for query results</li>
                <li><b>VoxelData:</b> Individual voxel data</li>
            </ul>
            """))
        elif nav_value == 'Workflows':
            display(HTML("""
            <h3>Common Workflows</h3>
            <p>Select a workflow from the right panel to see step-by-step guide.</p>
            <p>The framework supports various workflows:</p>
            <ul>
                <li><b>Complete Workflow:</b> End-to-end from query to visualization</li>
                <li><b>Quick Analysis:</b> Fast analysis workflow</li>
                <li><b>Quality Assessment:</b> Focused quality evaluation</li>
            </ul>
            """))
        elif nav_value == 'Installation':
            display(HTML(f"""
            <h3>Installation Status</h3>
            <p><b>AM-QADF Installed:</b> {'✅ Yes' if AM_QADF_AVAILABLE else '❌ No'}</p>
            <p><b>Version:</b> {AM_QADF_VERSION}</p>
            <p><b>MongoDB Available:</b> {'✅ Yes' if MONGODB_AVAILABLE else '❌ No'}</p>
            <h4>Dependencies:</h4>
            <ul>
                {'<li>✅ numpy</li>' if DEPENDENCIES['numpy'] else '<li>❌ numpy</li>'}
                {'<li>✅ pandas</li>' if DEPENDENCIES['pandas'] else '<li>❌ pandas</li>'}
                {'<li>✅ scipy</li>' if DEPENDENCIES['scipy'] else '<li>❌ scipy</li>'}
                {'<li>✅ pymongo</li>' if DEPENDENCIES['pymongo'] else '<li>❌ pymongo</li>'}
                {'<li>✅ matplotlib</li>' if DEPENDENCIES['matplotlib'] else '<li>❌ matplotlib (optional)</li>'}
                {'<li>✅ pyvista</li>' if DEPENDENCIES['pyvista'] else '<li>❌ pyvista (optional)</li>'}
                {'<li>✅ ipywidgets</li>' if DEPENDENCIES['ipywidgets'] else '<li>❌ ipywidgets</li>'}
            </ul>
            <h4>Installation Instructions:</h4>
            <pre>
# Clone repository
git clone &lt;repository-url&gt;
cd AM-QADF

# Install dependencies
pip install -r requirements.txt

# Install package
pip install -e .
            </pre>
            """))

quick_nav.observe(lambda change: update_center_panel(change['new']), names='value')

# Right Panel: Quick Reference
# Key Concepts
concepts_accordion = Accordion(children=[])
for concept_name, concept_info in KEY_CONCEPTS.items():
    concept_html = f"""
    <p><b>Definition:</b> {concept_info['definition']}</p>
    <p><b>Usage:</b> {concept_info['usage']}</p>
    """
    concepts_accordion.children += (widgets.HTML(concept_html),)
    concepts_accordion.set_title(len(concepts_accordion.children) - 1, concept_name)

# Data Structures
data_structure_dropdown = Dropdown(
    options=list(DATA_STRUCTURES.keys()),
    description='Data Structure:',
    style={'description_width': 'initial'}
)

data_structure_output = Output()

def update_data_structure(change):
    """Update data structure display."""
    with data_structure_output:
        clear_output(wait=True)
        struct_name = change['new']
        struct_info = DATA_STRUCTURES[struct_name]
        
        html = f"""
        <h4>{struct_name}</h4>
        <p><b>Description:</b> {struct_info['description']}</p>
        <p><b>Attributes:</b> {', '.join(struct_info['attributes'])}</p>
        <p><b>Example:</b></p>
        <pre>{struct_info['example']}</pre>
        """
        display(HTML(html))

data_structure_dropdown.observe(update_data_structure, names='value')
update_data_structure({'new': list(DATA_STRUCTURES.keys())[0]})

# Common Workflows
workflow_dropdown = Dropdown(
    options=list(WORKFLOWS.keys()),
    description='Workflow:',
    style={'description_width': 'initial'}
)

workflow_output = Output()

def update_workflow(change):
    """Update workflow display."""
    with workflow_output:
        clear_output(wait=True)
        workflow_name = change['new']
        workflow_info = WORKFLOWS[workflow_name]
        
        html = f"""
        <h4>{workflow_name}</h4>
        <p><b>Description:</b> {workflow_info['description']}</p>
        <p><b>Steps:</b></p>
        <ol>
        """
        for step in workflow_info['steps']:
            html += f"<li>{step}</li>"
        html += "</ol>"
        display(HTML(html))

workflow_dropdown.observe(update_workflow, names='value')
update_workflow({'new': list(WORKFLOWS.keys())[0]})

# Installation Status
install_status_html = f"""
<h4>Installation Status</h4>
<p><b>AM-QADF:</b> {'✅ Installed' if AM_QADF_AVAILABLE else '❌ Not Installed'}</p>
<p><b>MongoDB:</b> {'✅ Connected' if MONGODB_AVAILABLE else '❌ Not Connected'}</p>
<h5>Dependencies:</h5>
<ul>
    <li>{'✅' if DEPENDENCIES['numpy'] else '❌'} numpy</li>
    <li>{'✅' if DEPENDENCIES['pandas'] else '❌'} pandas</li>
    <li>{'✅' if DEPENDENCIES['scipy'] else '❌'} scipy</li>
    <li>{'✅' if DEPENDENCIES['pymongo'] else '❌'} pymongo</li>
    <li>{'✅' if DEPENDENCIES['matplotlib'] else '⚠️'} matplotlib (optional)</li>
    <li>{'✅' if DEPENDENCIES['pyvista'] else '⚠️'} pyvista (optional)</li>
</ul>
"""
install_status = widgets.HTML(install_status_html)

right_panel = VBox([
    widgets.HTML("<h3>Key Concepts</h3>"),
    concepts_accordion,
    widgets.HTML("<h3>Data Structures</h3>"),
    data_structure_dropdown,
    data_structure_output,
    widgets.HTML("<h3>Common Workflows</h3>"),
    workflow_dropdown,
    workflow_output,
    widgets.HTML("<h3>Installation Status</h3>"),
    install_status
], layout=Layout(width='350px', padding='10px', border='1px solid #ccc'))

# Bottom Panel: Status and Help
status_display = widgets.HTML("<b>Status:</b> Overview | <b>Help:</b> Click modules to explore")
help_text = widgets.HTML("Select a module from the left panel to see details. Use navigation dropdown to jump to sections.")

bottom_panel = VBox([
    status_display,
    help_text
], layout=Layout(padding='10px', border='1px solid #ccc'))

# Main Layout
left_panel = VBox([
    widgets.HTML("<h3>Module Explorer</h3>"),
    module_accordion
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

center_panel = VBox([
    center_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Initialize center panel
update_center_panel('Overview')

# Display the interface
display(main_layout)


## Summary

Congratulations! You've completed the introduction to AM-QADF.

### Key Takeaways

1. **Framework Purpose**: AM-QADF processes, analyzes, and visualizes multi-source additive manufacturing data
2. **Architecture**: Modular, layered architecture with 13 modules
3. **Core Concepts**: VoxelGrid, Signal Mapping, Data Fusion, Quality Assessment
4. **Workflows**: Supports various workflows from quick analysis to complete processing

### Next Steps

Proceed to:
- **01_Data_Query_and_Access.ipynb** - Query data from warehouse
- **02_Voxel_Grid_Creation.ipynb** - Create voxel grids
- **03_Signal_Mapping_Fundamentals.ipynb** - Master signal mapping

### Related Resources

- Framework Documentation: `../docs/AM_QADF/README.md`
- API Reference: `../docs/AM_QADF/06-api-reference/README.md`
- Examples: `../examples/README.md`
